### Graded Lab 5

Hello ! Welcome to Graded Lab of Module 5.

In the last assignment we had worked on missing values & outliers.
Its high time when we look at the feature engineering & Principal component analysis.

**So lets go ahead with the last graded assignment for the course !**

In case if you are not able to recollect the problem description and data description then mentioning it below.

Lets look at the problem statement,

*Client: ABC Retail, Incorporated, rest-of-the-world division* 

***Project name: Online retail sales analysis*** 

An online retailer, ABC, Inc., operates in nearly 100 countries worldwide, selling furniture, office supplies and technology products to customers in three segments: consumer, corporate and home office. ABC, Inc. is a US-based company, and it has two major divisions: US and rest of the world. We are working with the rest of the world division of the company. 

They have provided us with online sales transaction data from 2011 to 2014.

We are given 3 datasets:-

1. Data on each sale; 51290 records; all data in US dollars
It contains fields like
**order_id** (identifier) ,order_date ,ship_date ,ship_mode ,**customer_id**(identifier) ,product_id ,category ,sub_category ,product_name ,sales ,quantity ,discount ,profit ,shipping_cost ,order_priority ,**vendor_code** (identifier) 


2. Data on the customers; 1590 records 
It contains fields like
**customer_id** (identifier) ,customer_name ,city ,state ,country ,postal_code ,segment ,market ,region 

3. Data on vendors who supply the retailer; 65 records 
It contains fields like
vendor ,**vendor_code** (identifier) 

We need to analyze the data and need to provide answer to different questions asked by company officials.

In [1]:
# importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from itertools import combinations
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

### Reading sales data
sales = pd.read_csv('sales_data.csv')

### Reading customer data
cust = pd.read_csv(r'customers.csv',encoding='iso-8859-1')

### Reading vendor data
vend = pd.read_csv(r'vendors.csv')
sales.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,product_id,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,vendor_code
0,1,MX-2014-143658,02/10/2014,06/10/2014,Standard Class,SC-20575,OFF-LA-10002782,Office Supplies,Labels,"Hon File Folder Labels, Adjustable",13.08,3,0.0,4.56,1.03,Medium,VE_001
1,2,MX-2012-155047,15/10/2012,20/10/2012,Standard Class,KW-16570,FUR-FU-10004015,Furniture,Furnishings,"Tenex Clock, Durable",252.16,8,0.0,90.72,13.45,Medium,VE_002
2,3,MX-2012-155047,15/10/2012,20/10/2012,Standard Class,KW-16570,FUR-BO-10002352,Furniture,Bookcases,"Ikea 3-Shelf Cabinet, Mobile",193.28,2,0.0,54.08,9.63,Medium,VE_003
3,4,MX-2012-155047,15/10/2012,20/10/2012,Standard Class,KW-16570,OFF-BI-10004428,Office Supplies,Binders,"Cardinal Binder, Clear",35.44,4,0.0,4.96,1.37,Medium,VE_004
4,5,MX-2012-155047,15/10/2012,20/10/2012,Standard Class,KW-16570,OFF-AR-10004594,Office Supplies,Art,"Sanford Canvas, Water Color",71.60,2,0.0,11.44,3.79,Medium,VE_005


In [2]:
pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format','{:.2f}'.format)

import datetime
import warnings

warnings.filterwarnings("ignore")

In [3]:
sales.shape, cust.shape, vend.shape

((51290, 17), (1590, 9), (65, 2))

In [ ]:
cust.head()

In [ ]:
vend.head()

In order to solve the next questions , we need to combine all the 3 datasets into a single dataframe such that every details of sales dataframe are intact. So here we have written a data processing function.
There are 2 tasks which are to be performed.
1. Merge/ Join all the 3 datasets into a single dataframe such that every details of sales dataframe are intact. (Understand which should be the joining key , type of join , refer .merge() function of pandas)
2. Convert 'order_date' into a datetime column.
**Return output as a dataframe**

In [4]:
#### data_merging & order_date processing , data1 will be sales , data2 will be customer dataset & data3 will be vendor dataset.

def data_process(data1,data2,data3):
    # your code here
    df = data1.merge(right=data3, how="left", on="vendor_code")
    df2 = df.merge(right=data2, how="left", on="customer_id")
    df2["order_date"] = pd.to_datetime(df2['order_date'], format='%d/%m/%Y')
    return df2

In [5]:
sales= data_process(data1=sales.copy(),data2=cust.copy(),data3=vend.copy())

In [6]:
sales.shape

(51290, 26)

In [7]:
assert sales['order_date'].dtypes=='<M8[ns]' ,'Make sure if you have converted order_date into a datetime format correctly or not.'
assert sales.shape== (51290,26) ,'Checking size and shape of dataframe after merging is a very important check.'

### Q.1) Create a feature for identifying which orders are delivered on weekends. Flag will take value 1 if order is delivered on weekends & 0 otherwise. (Sat & Sun are weekeends). Return  the output series.

In [ ]:
sales.columns

In [ ]:
sales.head()

In [ ]:
sales['dayweek'] = sales['order_date'].dt.dayofweek

In [ ]:
sales.head()

In [ ]:
sales["dayweek"].value_counts()

In [ ]:
sales['delivered_on_weekend'] = sales['dayweek'].apply(lambda x:1 if x >=5 else 0)

In [ ]:
sales.head()

In [ ]:
sales['delivered_on_weekend'].value_counts()

In [8]:
def weekend_flag(data):
    
  
    # dt.dayoftheweek >= 5  # 5 and 6 represent Saturday and Sunday
    
    data['dayweek'] = data['order_date'].dt.dayofweek

    # Create a new column 'delivered_on_weekend' with 1 for weekend deliveries and 0 for others
    data['delivered_on_weekend'] = (data['dayweek'].apply(lambda x:1 if x >=5 else 0)).astype(int)
    
    return data['delivered_on_weekend']

In [9]:
sales['weekend_orders']=weekend_flag(data=sales)

In [10]:
# autograder cell , please do not alter/ delete /edit this cell
assert all(sales['weekend_orders'].unique()==np.array([0, 1])) ,'Make sure that you have created flag correctly'


### Q.2) Create a feature called 'discount_per_quantity' Return  the output series.

In [ ]:
sales.discount

In [ ]:
sales.quantity

In [ ]:
feature_1(data=sales,col1='discount',col2='quantity')

In [11]:
def feature_1(data,col1,col2):
    data['discount_per_quantity'] = data[col1] / data[col2]
    return data['discount_per_quantity']

In [12]:
sales['discount_per_quantity']= feature_1(data=sales,col1='discount',col2='quantity')

In [13]:
# autograder cell , please do not alter/ delete /edit this cell

### Q.3) Create a feature for finding cumulative sales for every customer. Return  the output series.

In [ ]:
sales.sort_values(['customer_id','order_date'],ascending=[1,1],inplace=True)

In [ ]:
sales.head()

In [ ]:
df = pd.DataFrame(sales.groupby("customer_id", as_index=False)["sales"].sum())

df

In [14]:
def cumulative_sales(data):
    df = data.groupby('customer_id')['sales'].cumsum()
    return df

In [15]:
### Sorting the dataframe wrto customer_id & order_date
sales.sort_values(['customer_id','order_date'],ascending=[1,1],inplace=True)
### Applying cumulative_sales function
sales['cumulative_sales']=cumulative_sales(data=sales)

In [16]:
# autograder cell , please do not alter/ delete /edit this cell

### Q.4) Please demonstrate how to enhance a given dataset by introducing a new feature that records the frequency of same-day purchases made by each customer. The desired outcome should be a modified dataframe containing a variable named 'same_day_purchase_frequency' which accurately portrays the same-day purchase behavior of individual customers.

In [ ]:
sales.head()

In [ ]:
order_counts = sales.groupby(['customer_id', sales['order_date'].dt.date])['order_id'].count().reset_index()

In [ ]:
order_counts

In [ ]:
order_counts.rename(columns={'order_id': 'order_count'}, inplace=True)

In [ ]:
same_day_purchases = order_counts[order_counts['order_count'] > 1]
same_day_purchases

In [ ]:
same_day_purchase_frequency = same_day_purchases.groupby('customer_id')['order_count'].mean().reset_index()
same_day_purchase_frequency.rename(columns={'order_count': 'same_day_purchase_frequency'}, inplace=True)
same_day_purchase_frequency

In [ ]:
sales = sales.merge(same_day_purchase_frequency, on='customer_id', how='left')

In [ ]:
sales.head()

In [17]:
def same_day_purchase(data):
    
    
    # Group by 'customer_id' and date, and count the orders, store count of orders in variable named 'order_count'
    order_counts = data.groupby(['customer_id', data['order_date'].dt.date])['order_id'].count().reset_index()
    order_counts.rename(columns={'order_id': 'order_count'}, inplace=True)
    
    # Filter for same-day purchases (order_count > 1)
    same_day_purchases = order_counts[order_counts['order_count'] > 1]
    
    
    # Group by 'customer_id' and calculate the same-day purchase frequency , name that column as 'same_day_purchase_frequency'
    same_day_purchase_frequency = same_day_purchases.groupby('customer_id')['order_count'].mean().reset_index()
    same_day_purchase_frequency.rename(columns={'order_count': 'same_day_purchase_frequency'}, inplace=True)
    
    # Merge the result with the original DataFrame to add the same-day purchase frequency as a new column  
    data = data.merge(same_day_purchase_frequency, on='customer_id', how='left')


    # Fill missing values with 0 (for customers with no same-day purchases)
    data['same_day_purchase_frequency'].fillna(0, inplace=True)

    return data

In [18]:
sales_new=same_day_purchase(data=sales.copy())
# autograder cell , please do not alter/ delete /edit this cell

### Q.5) Create a variable which will calculate weighted sales per region. Return  the output series.
Weights for every region are :-'LATAM':1, 'EMEA':1.3, 'Africa':2.7, 'Canada':1.4, 'EU':2.8, 'APAC':3.5

In [ ]:
sales.head()

In [ ]:
sales.market.value_counts()

In [ ]:
# Define a dictionary with region weights
region_weights = {
        'LATAM': 1,
        'EMEA': 1.3,
        'Africa': 2.7,
        'Canada': 1.4,
        'EU': 2.8,
        'APAC': 3.5
    }

In [ ]:
weighted_sales_calculation = lambda row: row['sales'] * region_weights.get(row['market'], 1)

In [ ]:
sales['weighted_sales'] = sales.apply(weighted_sales_calculation, axis=1)

In [ ]:
sales.head()

In [21]:
def weighted_sales(data):
    # Write a lambda function & apply it to dataframe.Kindly ignore raiseNotImplementError
    # Define a dictionary with region weights
    region_weights = {
        'LATAM': 1,
        'EMEA': 1.3,
        'Africa': 2.7,
        'Canada': 1.4,
        'EU': 2.8,
        'APAC': 3.5
    }
    
    # Define a lambda function to calculate weighted sales based on 'sales' and 'region' columns
    weighted_sales_calculation = lambda row: row['sales'] * region_weights.get(row['market'], 1)

    # Apply the lambda function to the DataFrame to calculate weighted sales
    data['weighted_sales'] = data.apply(weighted_sales_calculation, axis=1)
    
    return data

In [22]:
sales['weighted_market_sales']=weighted_sales(data=sales)

ValueError: Cannot set a DataFrame with multiple columns to the single column weighted_market_sales

In [23]:
# autograder cell , please do not alter/ delete /edit this cell

Most of the variables in our data are categorical in nature & for further anlysis purpose if we convert them by ordinal encoding or one hot encoding, we will be left with too many variables.
If we have too many variables in the data , it might cause trouble in later stages like machine learning model building.
1. Having too many variables can lead to overfitting and poor generalization to new data.
2. It can make it difficult to analyze and interpret the data and identify the most important variables.
3. It can increase computational complexity and time required to build and train a model.

How to deal with this ?

Wait , we have learned about **Principal component analysis** , we can leverage that method here for **dimensionality reduction.**

### Q.6 ) Complete the following code for PCA implementation. Return the no of PC's required for capturing the 80% of variation, variance explained by the PC's .

In [ ]:
sales.head()

In [ ]:
def PCA(data,target=0.80):
    # your code here

    # Treating ordinal variable ,
    # order_priority : 'Medium':2, 'Low':1, 'High':3, 'Critical':4 
    data['order_priority_encoded'] = data['order_priority'].map({'Low': 1, 'Medium': 2, 'High': 3, 'Critical': 4})

    
    # Treating missing values
    # Impute the missing values in 'order_prioirty_encoded','ship_mode' by mode, 
    data['order_priority_encoded'].fillna(data['order_priority_encoded'].mode()[0], inplace=True)
    data['ship_mode'].fillna(data['ship_mode'].mode()[0], inplace=True)

    
    # Treating Nominal variables ,'ship_mode','category','sub_category' (while encoding them make sure to select k-1 dummy variables for a variable with k categories)
    data = pd.get_dummies(data, columns=['ship_mode', 'category', 'sub_category'], drop_first=True)

    # Final data selection
    # Select all the variables with 'sales','quantity','discount','profit','shipping_cost','order_priority_encoded' & dummies in 'ship_mode','category','sub_category'.
    data_1 = data[['sales','quantity','discount','profit','shipping_cost','order_priority_encoded','ship_mode_Same Day','ship_mode_Second Class','ship_mode_Standard Class','category_Office Supplies','category_Technology','sub_category_Appliances','sub_category_Art','sub_category_Binders','sub_category_Bookcases','sub_category_Chairs','sub_category_Copiers','sub_category_Envelopes','sub_category_Fasteners','sub_category_Furnishings','sub_category_Labels','sub_category_Machines','sub_category_Paper','sub_category_Phones','sub_category_Storage','sub_category_Supplies','sub_category_Tables']]

    
    # Standardize the data
    sc = StandardScaler()
    
    cols = ['sales','quantity','discount','profit','shipping_cost','order_priority_encoded']
    for col in cols:
        data_1[col] = sc.fit_transform(data_1[cols])

    
    from sklearn.decomposition import PCA
    
    # Create a PCA instance    
    pca = PCA()
    
    # Fit PCA to the scaled data
    pca.fit(data_1)
    
    #  Calculate cumulative explained variance
    explained_variance_ratio = pca.explained_variance_ratio_
    cumulative_variance = np.cumsum(explained_variance_ratio)
    
    
    target_variation=target
    n_components = np.argmax(cumulative_variance >= target_variation) + 1
 
    
    # Excellent we got no of pcs which are required for capturing the target variation in the data
    
    ### Fitting PCA again with no of pc's we got above.
    pca = PCA(n_components=n_components)
    
    # Fit PCA to the scaled data
    pca.fit(data_1)
    
    #  Calculate cumulative explained variance
    explained_variance_ratio = pca.explained_variance_ratio_
       
    return n_components,explained_variance_ratio 

In [ ]:
# autograder cell , please do not alter/ delete /edit this cell
assert type(PCA(data=sales.copy())[0])==np.int64 , 'Number of components required for capturing target variation should be in integer format.'
assert type(PCA(data=sales.copy())[1])==np.ndarray , 'Type of variation captured should be in an array format.'

### Q.7) How much variation was explained by 3rd principal component ? 
**Round output to 4 digits**

In [ ]:
def var_expl(variation_explained):
    # Kindly ignore raiseNotImplementError   
    # your code here

In [ ]:
# autograder cell , please do not alter/ delete /edit this cell